In [2]:
import pandas as pd

In [3]:
spark

## Write dummy data

In [4]:
pdf = pd.DataFrame({'a' : [x for x in range(10)]})
df = spark.createDataFrame(pdf)

In [5]:
df.repartition(4).write.format('delta').save('./datasets/test.delta', mode = 'overwrite')

In [6]:
pdf = pd.DataFrame({'a' : [x for x in range(1_000)]})
df = spark.createDataFrame(pdf)

In [7]:
df.repartition(4).write.format('delta').save('./datasets/test.delta', mode = 'overwrite')

## Check delta time travel

In [8]:
import delta

In [9]:
dt = delta.DeltaTable.forPath(spark, path='./datasets/test.delta/')

In [10]:
dt.history().toPandas()

,version,timestamp,userId,userName,operation,operationParameters,job,notebook,clusterId,readVersion,isolationLevel,isBlindAppend,operationMetrics,userMetadata,engineInfo
0,3,2022-07-11 15:14:07.374,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,2.0,Serializable,False,"{'numOutputRows': '1000', 'numOutputBytes': '5...",None,Apache-Spark/3.2.1 Delta-Lake/1.2.1
1,2,2022-07-11 15:13:57.421,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,1.0,Serializable,False,"{'numOutputRows': '10', 'numOutputBytes': '830...",None,Apache-Spark/3.2.1 Delta-Lake/1.2.1
2,1,2022-07-11 13:55:55.164,None,None,WRITE,"{'mode': 'Overwrite', 'partitionBy': '[]'}",None,None,None,0.0,Serializable,False,"{'numOutputRows': '1000', 'numOutputBytes': '5...",None,Apache-Spark/3.2.1 Delta-Lake/1.2.1
3,0,2022-07-11 13:48:50.411,None,None,WRITE,"{'mode': 'ErrorIfExists', 'partitionBy': '[]'}",None,None,None,NaN,Serializable,True,"{'numOutputRows': '100', 'numOutputBytes': '23...",None,Apache-Spark/3.2.1 Delta-Lake/1.2.1


In [11]:
(
    spark
    .read
    .format('delta')
    .option('versionAsOf', 1)
    .load('./datasets/test.delta/').toPandas()
)

,a
0,19
1,43
2,60
3,46
4,38
...,...
995,940
996,934
997,951
998,971
